Now we're changing things up 
We needs to use our current data and based off what we already listened to find the songs in the categories we listened to lease and re recommend them basef on:
- users profile vector
- each songs profile vector 
- the clustering of groups 

"What behavioral patterns exist in personal listening data, and how can statistical modeling surface underexplored listening behaviors?”
“Within underexplored behavior clusters, which songs best match the user’s preference profile?”
---
Engagment Scalars:

play_count               → total plays
skip_rate                → skips / plays
completion_rate          → full listens / plays
avg_listen_ratio         → avg(seconds_listened / song_length)
replay_rate              → replays / plays
---
Temporal Behavior:

recency_score             → exp decay of last play time
time_of_day_distribution  → % morning / afternoon / night
day_type_distribution     → % weekday / weekend
---
Behavioral Context:

session_depth_avg         → avg position in session
repeat_interval_avg       → avg time between replays
---
Ex:
song_vector = [
    play_count_norm,
    skip_rate,
    completion_rate,
    avg_listen_ratio,
    replay_rate,
    recency_score,
    weekday_ratio,
    weekend_ratio,
    morning_ratio,
    evening_ratio,
    session_depth_avg_norm
]

In [30]:
#Imports 
import pandas as pd 
import matplotlib as plot 
import matplotlib.pyplot as plt
import sqlite3

In [31]:
# get data 
df = pd.read_csv(rf"Apple Music Play Activity.csv")

/var/folders/ms/q3g72bzn7hb3nqw2hgm84bq40000gp/T/ipykernel_8221/1500770189.py:2: DtypeWarning: Columns (0: Bundle Version, 1: Grouping, 2: House ID, 3: Local Radio Station ID, 4: Radio Station Country, 5: Radio Station ID, 6: Radio Type, 7: Radio User ID, 8: Recommended Content Localization Key, 9: Shelf Content Identifier, 10: Source Radio Name, 11: Source Radio Type, 12: Transition Type) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(rf"Apple Music Play Activity.csv")


In [32]:
#clean data 

# Step 1 — identify columns to drop // drop  where: null percentage is greater than or equal to 90% and where there is only no or 1 unique value
nulls = df.isna().mean().sort_values(ascending=False)
unique = df.nunique().sort_values()
drop_cols = nulls[(nulls >= 0.9) | (unique == 0) | (unique == 1)].index

# Step 2 — drop them
newdf = df.drop(columns=drop_cols)

# 3 - Drop all columns that have keywords in their name
keywords_to_drop = ["Container Origin","Container Album Name","Container Name","Milliseconds Since Play","Event Received Timestamp","Event Timestamp","Event Post Date","Evaluation Variant","Source Type","ID", "Client","Version","Device","Offline","IP","User's","Provided","Siri","Display","Use Listening", "Subscription", "Session Is", "Personalized","Ownership","Media Type","Media Bundle","Item Type","Vocal","Event Reason Hint","Repeat Play" ]
newdf = newdf.drop(columns=[col for col in newdf.columns if any(k in col for k in keywords_to_drop)])
# 4 - Drop rows where Song Name is null
newdf = newdf.dropna(subset=["Song Name"])
newdf = newdf[~((newdf['Start Position In Milliseconds'] == 0) & (newdf['End Position In Milliseconds'] == 0) | (newdf['End Position In Milliseconds'] == 0) | (newdf['End Position In Milliseconds']).isna())]
newdf = newdf[~((newdf['Media Duration In Milliseconds']==0) | (newdf['Media Duration In Milliseconds'].isna()))]

#get rid of null event dates rows
newdf=newdf[~(newdf['Event End Timestamp'].isna()|newdf['Event Start Timestamp'].isna())]
# --- Convert timestamps to datetime objects ---
newdf['Event Start Timestamp'] = pd.to_datetime(newdf['Event Start Timestamp'], utc=True, format="ISO8601")
newdf['Event End Timestamp'] = pd.to_datetime(newdf['Event End Timestamp'], utc=True, format="ISO8601")

# --- Convert durations to seconds ---
newdf['Media Duration Sec'] = newdf['Media Duration In Milliseconds'] / 1000
newdf['Play Duration Sec'] = newdf['Play Duration Milliseconds'] / 1000
newdf['Start Position Sec'] = newdf['Start Position In Milliseconds'] / 1000
newdf['End Position Sec'] = newdf['End Position In Milliseconds'] / 1000

# --- Calculate percent of song played ---
newdf['Percent Played'] = newdf['Play Duration Sec'] / newdf['Media Duration Sec']

# --- Extract hour of day and day of week from start timestamp ---
newdf['Hour of Day'] = newdf['Event Start Timestamp'].dt.hour
newdf['Day of Week'] = newdf['Event Start Timestamp'].dt.day_name()

# flag skipped vs finished (e.g., <70% = skipped)
newdf['Skipped'] = newdf['Percent Played'] < 0.7

#Dropping any columns with milliseconds since we have seconds
newdf = newdf.drop(columns=[col for col in newdf.columns if any(k in col for k in ["Milliseconds"])])

#drop rows where Percent Played is grater than seconds played because that means we played the song longer than the song was which is impossible
newdf = newdf[newdf["Percent Played"]<=1]

We cleaned the data! We need to identify the features that are important to helping classify our behavioral patters and turn them into vectors:
(We need our personal profile vector, Each songs profile vector, and the clusters vectors)
- average listening time 
- Day listen to music the most 
- Hour I listen to music the most 
- Average Media Duration I listen to 
- Which Feature Name (place I listen to music the longest: album, playlist, random, etc) 
- Song name 

In [ ]:
#create user profile vector 
#average out all my songs and behaviors to make my profile 
# play_count               → total plays
# skip_rate                → skips / plays
# completion_rate          → full listens / plays
# avg_listen_ratio         → avg(seconds_listened / song_length)
# replay_rate              → replays / plays

UserVector =[]
play_count = len(newdf)
skip_rate = len(newdf[newdf["Skipped"]==True]) / play_count
completion_rate = len(newdf[newdf["Skipped"]==False]) / play_count
avg_listen_ratio = (
    newdf.loc[newdf["Skipped"] == False, "Play Duration Sec"] /
    newdf.loc[newdf["Skipped"] == False, "Media Duration Sec"]
).mean()
#out of all my songs how many times did I replay any song 
replay_rate = sum(newdf["Song Name"].value_counts()-1) / play_count

UserVector.extend([play_count,skip_rate,completion_rate,avg_listen_ratio,replay_rate])


[178341, 0.5179739936413948, 0.48202600635860515, np.float64(0.9831011152237121), 0.9257882371412071]


In [ ]:
#create song profile 
#for every song make a vector which is already just their row 
unique_songs = newdf["Song Name"].unique()
unique_songs_counts = newdf["Song Name"].value_counts()
song_vector_list =[] #array of all the completed song vectors 
for song in unique_songs:
    SongVector =[]
    play_count = unique_songs_counts[song]
    skip_rate = len(newdf[(newdf["Song Name"]==song) & (newdf["Skipped"]==True)]) / play_count
    completion_rate = len(newdf[(newdf["Song Name"]==song) & (newdf["Skipped"]==False)]) / play_count
    avg_listen_ratio = (
        newdf.loc[(newdf["Skipped"] == False) & (newdf["Song Name"]==song), "Play Duration Sec"] /
        newdf.loc[(newdf["Skipped"] == False) & (newdf["Song Name"]==song), "Media Duration Sec"]
    ).mean()
    SongVector.extend([play_count,skip_rate,completion_rate,avg_listen_ratio,replay_rate])
    song_vector_list.append(SongVector)
print(len(song_vector_list))




13235


Step B — Normalize vectors
(important so play_count doesn’t dominate)
StandardScale

We are going to use unsupervised to see what groupings I can get for my data, and then use an distance functions to score them based off of our song's vectors proximity to our user's vectors.

Once again: we want to know based on our song vector profile and our songs profiles which song to re-recommend for a listen that would resonate with us the most? 

We will use a score to do this. 

How will we compute the score though? well the model could possibly compute the score and then return our scores and we just get the scores that have the higest match rate. 

If we do that then how will we rank each scalar in our vector and weight it? 

Let's think....
Every song vector has a: play_count, skip_rate, completion_rate, and a avg_listen_ratio. When playing a song what matters most in terms of being similar to our users: play_count, skip_rate, completion_rate, avg_listen_ratio, and replay_rate?

well we could take a tensor mapping of all of our vectors and see how close they are to the users and use the distance as a measurement for score, then get all the songs with one play and get only the points that have the smallest distace from the users vector point and return them. like top 5.







In [ ]:
#create clusters and evaluate using different clustering algorithims to find behaviors that influence// unsupervised 
from sklearn import cluster,neighbors,naive_bayes
import sklearn

#KNN
neighbors.KNeighborsClassifier
neighbors.KNeighborsRegressor

#Kmeans 
cluster.k_means()
#DBSCAN
cluster.dbscan()
#Agglomerative Clustering
cluster.AgglomerativeClustering

Distance functions are what we'll use to see how far our song vectors are from our users vector. we can use any of the three distance measures below:
1) Euclidean distance
d = ||U - S||
2) Cosine similarity (very common)
score = cos(θ) = (U · S) / (||U|| ||S||)
3) Manhattan distance
sum(|U_i - S_i|)
These are valid, standard, mathematically sound.

In [ ]:
#create scoring 
#scoring based on distance in tensor space 